# This Notebook will be used to complete Courseras Capstone project. 
## Than you for reading.

In [1]:
import pandas as pd
import numpy as np
import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [2]:
filename= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(filename) #downloading the data

In [3]:
toronto_data=df[0] #loading the table into a dataframe 

In [4]:
toronto_data.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
toronto_data.shape # Original table before the data wrangling

(180, 3)

In [6]:
toronto_data['Borough'].replace("Not assigned", np.nan, inplace = True) #replace all the Borough that are Not Assigned with NaN

In [7]:
toronto_data.head() #showing the results

,Postal Code,Borough,Neighbourhood
0,M1A,NaN,Not assigned
1,M2A,NaN,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
toronto_data.dropna(subset=["Borough"], axis=0, inplace=True) #dropping all the rows with NaN

In [9]:
toronto_data.shape #shows that we have dropped 77 rows

(103, 3)

In [10]:
toronto_data['Postal Code'].value_counts().max() #shows us that all the values in "Postal Code" are unique 

1

In [11]:
toronto_data['Neighbourhood'].str.count('Not assigned').max() #shows there are no Neighbourhood's that have the value 'Not assigned'

0

In [12]:
toronto_data.shape #The final shape of the df

(103, 3)

In [20]:
Coordinates = pd.read_csv('http://cocl.us/Geospatial_data') #Loading the Geospatial data

In [21]:
Coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
Coordinates.shape #checking that the dataframes match

(103, 3)

In [29]:
toronto_data_merge = pd.merge(toronto_data, Coordinates, on = 'Postal Code') #merging the 2 dataframes according to the postal codes

In [40]:
toronto_data_merge.describe() #checking that all the the rows have postal codes assigned 

,Latitude,Longitude
count,103.000000,103.000000
mean,43.704608,-79.397153
std,0.052463,0.097146
min,43.602414,-79.615819
25%,43.660567,-79.464763
50%,43.696948,-79.388790
75%,43.745320,-79.340923
max,43.836125,-79.160497


In [85]:
toronto_data_merge

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [86]:
#Creating a Folium map of Toronto with all the Neigborhoods visualized
map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=10)

for lat, lng, borough, neighborhood in zip(toronto_data_merge['Latitude'], toronto_data_merge['Longitude'], toronto_data_merge['Borough'], toronto_data_merge['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto